# Data Cleaning: T-ECD E-Commerce Transaction Dataset

## Project Context

This notebook documents the data cleaning process I performed on the T-ECD (Transactional E-Commerce Dataset) following exploratory data analysis.

## Methodological Note

- **Confirmed data errors**: Issues with direct evidence (e.g., missing value counts, duplicate counts, error messages)
- **Assumptions**: Choices I made in the absence of complete information (e.g., imputation strategies)
- **Open questions**: Uncertainties that remain unresolved (e.g., root causes of missing data)

## Dataset Overview (as of 2025-12-21)

The analysis revealed the following structure:

| Table | Rows | Key Issues Identified |
|-------|------|----------------------|
| Users | 3.5M | Missing demographics (1.7% region, 0.1% cluster) |
| Brands | 24.5K | Schema error in embeddings, 46 duplicates |
| Retail Items | 250K | Missing categories (3.8%), prices (10.6%) |
| Retail Events | 1.9M | No issues detected |
| Marketplace Items | 2.3M | Schema error, missing values unknown (file load failed in analysis) |
| Marketplace Events | 2.6M | Missing subdomain (0.04%) |
| Offers Items | 22K | Missing brand_id (2.4%) |
| Offers Events | 14.8M | No issues detected |
| Reviews | 11.5K | Schema error in embeddings |
| Payments Events | 11.5M | Missing brand_id (57.3%), price (0.004%) |
| Payments Receipts | 1.1M | Missing brand_id (90%), price (1.3%) |

In [ ]:
# Standard library
import sys
import io

# Fix console encoding for Windows systems
if sys.platform == 'win32':
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')

# Data manipulation
import pandas as pd
import numpy as np

# File operations
import os
from collections import defaultdict

# Data loading
from huggingface_hub import hf_hub_download, list_repo_files

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

## Configuration

In [ ]:
# Dataset repository
REPO_ID = "t-tech/T-ECD"
REPO_TYPE = "dataset"

# Local paths
CACHE_DIR = "dataset_cache"
OUTPUT_DIR = "cleaned_data"
DATASET_PATH_SMALL = "dataset/small"
DATASET_PATH_FULL = "dataset/full"

# Partition limits to manage memory
# I set these conservatively to ensure the notebook runs on typical hardware
DATASET_SMALL_NUM_PARTITIONS_TO_LOAD = 5
DATASET_FULL_NUM_PARTITIONS_TO_LOAD = 1

# Ensure output directories exist
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Helper Functions

Helper functions to standardize the cleaning workflow. These functions handle:
1. **Loading**: Parquet loading with fallback logic for schema errors
2. **Evidence display**: Showing concrete examples of data quality issues before cleaning
3. **Validation**: Quantifying the impact of cleaning operations
4. **Persistence**: Saving cleaned datasets for downstream use

In [ ]:
def load_remote_parquet_safe(filename, columns_to_exclude=None):
    """
    Loads a parquet file from Hugging Face, handling schema errors gracefully.
    
    Args:
        filename: Path to parquet file in the repo
        columns_to_exclude: List of columns to skip (e.g., corrupted embeddings)
    
    Returns:
        DataFrame or None if loading fails
    """
    print(f"Loading {filename}...")
    try:
        local_path = hf_hub_download(
            repo_id=REPO_ID,
            filename=filename,
            repo_type=REPO_TYPE,
            local_dir=CACHE_DIR,
            local_dir_use_symlinks=False
        )
        
        if columns_to_exclude:
            import pyarrow.parquet as pq
            schema = pq.read_schema(local_path)
            all_cols = [name for name in schema.names if not name.startswith('__')]
            use_cols = [c for c in all_cols if c not in columns_to_exclude]
            print(f"  Excluding columns: {columns_to_exclude}")
            print(f"  Reading columns: {use_cols}")
            df = pd.read_parquet(local_path, columns=use_cols)
            return df
        
        # Try loading all columns
        try:
            df = pd.read_parquet(local_path)
            return df
        except Exception as e:
            print(f"  Standard load failed: {e}")
            import pyarrow.parquet as pq
            schema = pq.read_schema(local_path)
            all_cols = [name for name in schema.names if not name.startswith('__')]
            if 'embedding' in all_cols:
                print(f"  Retrying without 'embedding' column...")
                use_cols = [c for c in all_cols if c != 'embedding']
                df = pd.read_parquet(local_path, columns=use_cols)
                print("  Success (with exclusions).")
                return df
            else:
                raise e
    except Exception as e:
        print(f"Error downloading/loading {filename}: {e}")
        return None


def load_dataframe_from_partitions_safe(file_list, limit=None, columns_to_exclude=None):
    """
    Loads and concatenates multiple parquet partition files.
    
    Args:
        file_list: List of partition file paths
        limit: Maximum number of partitions to load (None = all)
        columns_to_exclude: Columns to skip in all partitions
    
    Returns:
        Concatenated DataFrame or None
    """
    if not file_list:
        print("No files to load.")
        return None
    
    files_to_load = file_list[:limit] if limit else file_list
    print(f"Loading {len(files_to_load)} partitions (out of {len(file_list)} available)...")
    
    dfs = []
    for f in files_to_load:
        df = load_remote_parquet_safe(f, columns_to_exclude=columns_to_exclude)
        if df is not None:
            dfs.append(df)
    
    if not dfs:
        print("No dataframes loaded successfully.")
        return None
    
    print("Concatenating partitions...")
    full_df = pd.concat(dfs, ignore_index=True)
    return full_df


def show_evidence(df, name, columns_to_check=None):
    """
    Display evidence of data quality issues before cleaning.
    
    Args:
        df: DataFrame to examine
        name: Dataset name for reporting
        columns_to_check: Specific columns to highlight (None = all)
    """
    print(f"\n{'='*80}")
    print(f"EVIDENCE: {name}")
    print(f"{'='*80}")
    
    print(f"\nShape: {df.shape}")
    
    # Missing values
    missing = df.isnull().sum()
    missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
    
    if missing.sum() > 0:
        print(f"\n Missing Values Found:")
        missing_df = pd.DataFrame({
            'Column': missing.index,
            'Missing Count': missing.values,
            'Missing %': missing_pct.values
        })
        missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
        print(missing_df.to_string(index=False))
        
        # Show sample rows with missing values for key columns
        if columns_to_check:
            for col in columns_to_check:
                if col in df.columns and df[col].isnull().sum() > 0:
                    print(f"\n  Sample rows with missing '{col}':")
                    print(df[df[col].isnull()].head(2).to_string())
    else:
        print(f"\nNo missing values detected")
    
    # Duplicates (skip unhashable columns)
    try:
        hashable_cols = [c for c in df.columns 
                        if df[c].dtype != 'object' or 
                        not df[c].apply(lambda x: isinstance(x, (list, np.ndarray))).any()]
        dupes = df[hashable_cols].duplicated().sum()
        if dupes > 0:
            print(f"\nDuplicates Found: {dupes} rows")
        else:
            print(f"\nNo duplicates detected")
    except:
        print(f"\nDuplicate check skipped (unhashable columns)")
    
    print(f"\n{'='*80}\n")


def validate_cleaning(df_before, df_after, name):
    """
    Validate the impact of cleaning operations.
    
    Args:
        df_before: DataFrame before cleaning
        df_after: DataFrame after cleaning
        name: Dataset name for reporting
    """
    print(f"\n{'='*80}")
    print(f"VALIDATION: {name}")
    print(f"{'='*80}")
    
    rows_dropped = len(df_before) - len(df_after)
    print(f"\nRows: {len(df_before):,} → {len(df_after):,} (Dropped: {rows_dropped:,})")
    
    missing_before = df_before.isnull().sum().sum()
    missing_after = df_after.isnull().sum().sum()
    print(f"Missing Values: {missing_before:,} → {missing_after:,}")
    
    # Duplicates
    try:
        hashable_cols_before = [c for c in df_before.columns 
                               if df_before[c].dtype != 'object' or 
                               not df_before[c].apply(lambda x: isinstance(x, (list, np.ndarray))).any()]
        hashable_cols_after = [c for c in df_after.columns 
                              if df_after[c].dtype != 'object' or 
                              not df_after[c].apply(lambda x: isinstance(x, (list, np.ndarray))).any()]
        
        dupes_before = df_before[hashable_cols_before].duplicated().sum()
        dupes_after = df_after[hashable_cols_after].duplicated().sum()
        print(f"Duplicates: {dupes_before:,} → {dupes_after:,}")
    except:
        print(f"Duplicates: (skipped - unhashable columns)")
    
    if missing_after == 0 and rows_dropped >= 0:
        print(f"\nCLEANING SUCCESSFUL")
    else:
        print(f"\nWARNING: Review cleaning results")
    
    print(f"\n{'='*80}\n")


def save_cleaned_data(df, name):
    """
    Save cleaned dataset to parquet file.
    
    Args:
        df: Cleaned DataFrame
        name: Dataset name for filename
    """
    output_file = os.path.join(OUTPUT_DIR, f"{name.lower().replace(' ', '_')}_clean.parquet")
    df.to_parquet(output_file, index=False)
    print(f"Saved: {output_file} ({len(df):,} rows)")

## Indexing the Dataset

In [ ]:
print("Indexing remote dataset files...")
all_files = list_repo_files(repo_id=REPO_ID, repo_type=REPO_TYPE)
dataset_files = defaultdict(list)

for f in all_files:
    if f.endswith(".pq"):
        dirname = os.path.dirname(f).replace("\\", "/")
        dataset_files[dirname].append(f)

print(f"Indexed {len(dataset_files)} directories containing {sum(len(files) for files in dataset_files.values())} parquet files")

---

## 1. Users Table

### Evidence from Analysis

In `analysis.ipynb`:
- `socdem_cluster`: 5,153 missing values (0.1%)
- `region`: 58,917 missing values (1.7%)
- No duplicate records

The missing values are legitimate gaps in the demographic data—some users have unknown sociodemographic clusters or regions.

### Decision: Imputation Strategy

- To handle missing demographic data, dropping 58,917 users (1.7% of the base) was rejected because it would break foreign key relationships and discard behavioral data. 
- Imputing with the statistical mode was rejected due to potential bias, as missing data may be non-random and represent systematically different users. 
- Instead, we impute missing values with a sentinel value of -1, which preserves all records, maintains referential integrity, and allows downstream analyses to handle or exclude unknown demographics. The sentinel value is safe, as all original demographic codes are non-negative.

In [ ]:
# Load users table
print("Loading users table...")
users_path = f"{DATASET_PATH_SMALL}/users.pq"
df_users = load_remote_parquet_safe(users_path)

if df_users is not None:
    # Display evidence of data quality issues
    show_evidence(df_users, "Users (Before Cleaning)", 
                 columns_to_check=['socdem_cluster', 'region'])
    
    # Apply cleaning: impute missing demographics with -1
    print("Applying imputation strategy: missing demographics -> -1")
    df_users_clean = df_users.copy()
    df_users_clean['socdem_cluster'] = df_users_clean['socdem_cluster'].fillna(-1)
    df_users_clean['region'] = df_users_clean['region'].fillna(-1)
    
    # Validate cleaning impact
    validate_cleaning(df_users, df_users_clean, "Users")
    
    # Verify complete resolution
    assert df_users_clean.isnull().sum().sum() == 0, "Cleaning failed: missing values remain"
    
    # Save cleaned table
    save_cleaned_data(df_users_clean, "users")
    print(f"Saved {len(df_users_clean):,} cleaned user records")

---

## 2. Brands Table

### Evidence from Analysis

In `analysis.ipynb`:
1. **Schema error**: `embedding` column failed to load with error "Expected all lists to be of size=300 but index 1 had size=0"
2. **46 duplicate brand_id entries**: Violates the expected primary key constraint

The embedding column contains 300-dimensional vector representations of brands, intended for similarity calculations in recommendation systems. However, the source data contains inconsistent list lengths (empty lists mixed with 300-element lists), causing standard PyArrow loaders to fail schema validation before data can be accessed.

### Decision: Embedding Column Strategy

- Attempting low-level PyArrow recovery was rejected due to operational complexity and uncertain benefit, while replacing missing embeddings with dummy zero vectors was rejected because it would mislead models into treating them as legitimate brand representations. 
- Instead, the embedding column is excluded, preserving brand_id metadata and explicitly acknowledging the absence of embeddings. Downstream analyses must obtain embeddings from an alternative source or proceed without brand similarity features.

### Decision: Duplicate Resolution

- Among 46 duplicate brand_id values, manual investigation was rejected as unscalable without domain knowledge. 
- Keeping the first occurrence was chosen to ensure deterministic, reproducible behavior, prioritizing uniqueness over which specific duplicate is retained. Differences between duplicate records are assumed negligible, given minor metadata variations and the lack of additional identifying information.

In [ ]:
# Load brands table (exclude corrupted embedding column)
print("Loading brands table...")
brands_path = f"{DATASET_PATH_SMALL}/brands.pq"
df_brands = load_remote_parquet_safe(brands_path, columns_to_exclude=['embedding'])

if df_brands is not None:
    # Display evidence
    show_evidence(df_brands, "Brands (Before Cleaning)")
    
    # Show duplicate evidence explicitly
    duplicates_count = df_brands.duplicated(subset=['brand_id']).sum()
    print(f"\nDuplicate brand_id records: {duplicates_count}")
    if duplicates_count > 0:
        print("Sample duplicates:")
        duplicate_ids = df_brands[df_brands.duplicated(subset=['brand_id'], keep=False)]['brand_id'].unique()[:3]
        print(df_brands[df_brands['brand_id'].isin(duplicate_ids)].sort_values('brand_id'))
    
    # Remove duplicates
    print("\nApplying deduplication: keeping first occurrence of each brand_id")
    df_brands_clean = df_brands.drop_duplicates(subset=['brand_id'], keep='first').copy()
    
    # Validate
    validate_cleaning(df_brands, df_brands_clean, "Brands")
    save_cleaned_data(df_brands_clean, "brands")
    print(f"Saved {len(df_brands_clean):,} unique brand records")

---

## 3. Retail Items Table

### Evidence from Analysis

From `analysis.ipynb`:
- `category`: 9,585 missing (3.8%)
- `subcategory`: 9,585 missing (3.8%)
- `price`: 26,489 missing (10.6%)

Category and subcategory missingness is perfectly correlated—when one is missing, both are missing. This suggests these items were not fully cataloged during data collection.

### Decision: Category/Subcategory Strategy

- Dropping 3.8% of the catalog was rejected to avoid unnecessarily reducing analytic scope.
- Missing categorical attributes are imputed with the explicit label "Unknown" rather than dropping records or using mode values. This allows product catalog analyses to group "Unknown" items separately while retaining price and brand information, enabling revenue analysis even without category data. 

### Decision: Price Strategy

- Rows with missing prices are dropped rather than imputed. 
- Imputation would introduce severe bias due to diverse product categories and missing category labels—mean or median values would misrepresent items like low-cost food or high-cost home improvement products. Price is critical for revenue, affordability, and pricing analyses, so retaining imputed prices would compromise validity. 
- The resulting 10.6% data loss is an acceptable quality-coverage trade-off, prioritizing analytic integrity for research and model training, though a production system might make a different choice.

In [ ]:
# Load retail items
print("Loading retail items...")
retail_items_path = f"{DATASET_PATH_SMALL}/retail/items.pq"
df_retail_items = load_remote_parquet_safe(retail_items_path)

if df_retail_items is not None:
    show_evidence(df_retail_items, "Retail Items (Before Cleaning)",
                 columns_to_check=['category', 'subcategory', 'price'])
    
    # Apply cleaning
    print("Applying cleaning strategy:")
    print("  - category/subcategory: impute 'Unknown'")
    print("  - price: drop rows with missing values")
    
    df_retail_clean = df_retail_items.copy()
    df_retail_clean['category'] = df_retail_clean['category'].fillna("Unknown")
    df_retail_clean['subcategory'] = df_retail_clean['subcategory'].fillna("Unknown")
    df_retail_clean = df_retail_clean.dropna(subset=['price'])
    
    validate_cleaning(df_retail_items, df_retail_clean, "Retail Items")
    save_cleaned_data(df_retail_clean, "retail_items")
    print(f"Retained {len(df_retail_clean):,} items ({len(df_retail_clean)/len(df_retail_items)*100:.1f}% of original catalog)")

---

## 4. Retail Events Table

### Evidence from Analysis

From `analysis.ipynb`: No data quality issues detected. All columns complete, no duplicates.

### Decision: No Cleaning Required

In [ ]:
# Load and validate retail events
print("Loading retail events (validation only)...")
retail_events_dir = f"{DATASET_PATH_SMALL}/retail/events"
retail_events_files = dataset_files.get(retail_events_dir, [])
df_retail_events = load_dataframe_from_partitions_safe(
    retail_events_files, 
    limit=DATASET_SMALL_NUM_PARTITIONS_TO_LOAD
)

if df_retail_events is not None:
    show_evidence(df_retail_events, "Retail Events (Validation)")
    assert df_retail_events.isnull().sum().sum() == 0, "Unexpected data quality issues"
    print("Validation passed: no cleaning required")
    save_cleaned_data(df_retail_events, "retail_events")

---

## 5. Marketplace Items Table

### Evidence from Analysis

From `analysis.ipynb`:
- `subcategory`: 1,233,023 missing (53%)
- `category`: 966,395 missing (42%)  
- `price`: 2,882 missing (0.12%)

### Decision: Imputation Strategy

- For marketplace items, missing categorical attributes are imputed as "Unknown," consistent with the Retail Items strategy. Missing prices, which affect only 0.12% of records, are imputed with -1 rather than dropped. 
- The lower missingness shifts the coverage-quality trade-off: dropping such a small fraction has minimal impact, but imputation maximizes data retention for the larger marketplace catalog.

In [ ]:
# Load marketplace items
print("Loading marketplace items (embedding excluded due to schema error)...")
mp_items_path = f"{DATASET_PATH_SMALL}/marketplace/items.pq"
df_mp_items = load_remote_parquet_safe(mp_items_path, columns_to_exclude=['embedding'])

if df_mp_items is not None:
    show_evidence(df_mp_items, "Marketplace Items")
    
    # Apply cleaning if needed
    missing_count = df_mp_items.isnull().sum().sum()
    if missing_count > 0:
        print(f"Discovered {missing_count} missing values (not in original analysis)")
        print("Applying imputation strategy")
        
        df_mp_items_clean = df_mp_items.copy()
        for col in df_mp_items.columns:
            if df_mp_items[col].isnull().sum() > 0:
                if df_mp_items[col].dtype == 'object':
                    df_mp_items_clean[col] = df_mp_items_clean[col].fillna("Unknown")
                else:
                    df_mp_items_clean[col] = df_mp_items_clean[col].fillna(-1)
        
        validate_cleaning(df_mp_items, df_mp_items_clean, "Marketplace Items")
        save_cleaned_data(df_mp_items_clean, "marketplace_items")
    else:
        print("No cleaning required")
        save_cleaned_data(df_mp_items, "marketplace_items")

---

## 6. Marketplace Events Table

### Evidence from Analysis

From `analysis.ipynb`:
- `subdomain`: 1,115 missing (0.04%)
- All other columns complete

### Decision: Imputation Strategy

I impute missing subdomain values with "Unknown" to preserve complete event logs for user journey analysis. Events represent user actions and are valuable regardless of the subdomain context. Dropping 1,115 event records would create gaps in behavioral sequence data.

In [ ]:
# Load marketplace events
print("Loading marketplace events...")
mp_events_dir = f"{DATASET_PATH_SMALL}/marketplace/events"
mp_events_files = dataset_files.get(mp_events_dir, [])
df_mp_events = load_dataframe_from_partitions_safe(
    mp_events_files, 
    limit=DATASET_SMALL_NUM_PARTITIONS_TO_LOAD
)

if df_mp_events is not None:
    show_evidence(df_mp_events, "Marketplace Events (Before Cleaning)", 
                 columns_to_check=['subdomain'])
    
    # Apply cleaning
    print("Imputing missing subdomain with 'Unknown'")
    df_mp_events_clean = df_mp_events.copy()
    df_mp_events_clean['subdomain'] = df_mp_events_clean['subdomain'].fillna("Unknown")
    
    # Validate
    validate_cleaning(df_mp_events, df_mp_events_clean, "Marketplace Events")
    save_cleaned_data(df_mp_events_clean, "marketplace_events")
    print(f"Saved {len(df_mp_events_clean):,} event records")

---

## 7. Offers Items Table

### Evidence from Analysis

From `analysis.ipynb`:
- `brand_id`: 542 missing (2.4%)
- All other columns complete

### Decision: Imputation Strategy

I impute missing brand_id values with -1 rather than dropping records. Some offers may represent unbranded promotions or platform-wide deals that do not map to specific brands. The 2.4% missingness is modest, and offer-level attributes (item_id, offer terms) remain valid for promotion effectiveness analysis independent of brand attribution.

In [ ]:
# Load offers items
print("Loading offers items...")
offers_items_path = f"{DATASET_PATH_SMALL}/offers/items.pq"
df_offers_items = load_remote_parquet_safe(offers_items_path)

if df_offers_items is not None:
    show_evidence(df_offers_items, "Offers Items (Before Cleaning)", 
                 columns_to_check=['brand_id'])
    
    # Apply cleaning
    print("Imputing missing brand_id with -1")
    df_offers_items_clean = df_offers_items.copy()
    df_offers_items_clean['brand_id'] = df_offers_items_clean['brand_id'].fillna(-1)
    
    # Validate
    validate_cleaning(df_offers_items, df_offers_items_clean, "Offers Items")
    save_cleaned_data(df_offers_items_clean, "offers_items")
    print(f"Saved {len(df_offers_items_clean):,} offer records")

---

## 8. Offers Events Table

### Evidence from Analysis

From `analysis.ipynb`:
- No missing values detected
- No duplicate records

### Decision: No Cleaning Required

In [ ]:
# Load and validate offers events
print("Loading offers events (validation only)...")
offers_events_dir = f"{DATASET_PATH_SMALL}/offers/events"
offers_events_files = dataset_files.get(offers_events_dir, [])
df_offers_events = load_dataframe_from_partitions_safe(
    offers_events_files, 
    limit=DATASET_SMALL_NUM_PARTITIONS_TO_LOAD
)

if df_offers_events is not None:
    show_evidence(df_offers_events, "Offers Events (Validation)")
    
    # Assertion: verify no issues present
    assert df_offers_events.isnull().sum().sum() == 0, "Unexpected missing values detected"
    
    print("Validation passed: no cleaning required")
    save_cleaned_data(df_offers_events, "offers_events")
    print(f"Saved {len(df_offers_events):,} event records")

---

## 9. Reviews Table

### Evidence from Analysis

From `analysis.ipynb`:
- Embedding column has schema error (excluded during load)
- All data columns (timestamp, user_id, brand_id, rating) complete
- No duplicate records

### Decision: No Cleaning Required

In [ ]:
# Load and validate reviews (exclude corrupted embeddings)
print("Loading reviews (embedding column excluded due to schema error)...")
reviews_dir = f"{DATASET_PATH_SMALL}/reviews"
review_files = dataset_files.get(reviews_dir, [])
df_reviews = load_dataframe_from_partitions_safe(
    review_files, 
    limit=DATASET_SMALL_NUM_PARTITIONS_TO_LOAD, 
    columns_to_exclude=['embedding']
)

if df_reviews is not None:
    show_evidence(df_reviews, "Reviews (Validation)")
    
    # Verify completeness
    missing_count = df_reviews.isnull().sum().sum()
    if missing_count == 0:
        print("Validation passed: no missing values in data columns")
        save_cleaned_data(df_reviews, "reviews")
        print(f"Saved {len(df_reviews):,} review records")
    else:
        print(f"Warning: Unexpected {missing_count} missing values detected")

---

## 10. Payments Events Table

### Evidence from Analysis

From `analysis.ipynb`:
- `brand_id`: 6,583,207 missing (57.3%)
- `price`: 462 missing (0.004%)

### Decision: Differential Strategy

**For brand_id (57% missing):** I impute with -1 rather than dropping records. The extraordinarily high missingness likely reflects business logic—transactions may involve bundled items, generic products, or services without individual brand attribution. Dropping 57% of payment data would destroy the revenue analysis capability.

**For price (0.004% missing):** I drop rows with missing prices. Price is critical for payment analysis, and the 462 affected records represent negligible coverage loss (0.004%).

In [ ]:
# Load payments events
print("Loading payments events...")
pay_events_dir = f"{DATASET_PATH_FULL}/payments/events"
pay_events_files = dataset_files.get(pay_events_dir, [])
df_pay_events = load_dataframe_from_partitions_safe(
    pay_events_files, 
    limit=DATASET_FULL_NUM_PARTITIONS_TO_LOAD
)

if df_pay_events is not None:
    show_evidence(df_pay_events, "Payments Events (Before Cleaning)", 
                 columns_to_check=['brand_id', 'price'])
    
    # Apply cleaning
    print("Applying differential strategy:")
    print("  - brand_id (57% missing): impute with -1")
    print("  - price (0.004% missing): drop rows")
    
    df_pay_events_clean = df_pay_events.copy()
    df_pay_events_clean['brand_id'] = df_pay_events_clean['brand_id'].fillna(-1)
    df_pay_events_clean = df_pay_events_clean.dropna(subset=['price'])
    
    # Validate
    validate_cleaning(df_pay_events, df_pay_events_clean, "Payments Events")
    save_cleaned_data(df_pay_events_clean, "payments_events")
    print(f"Retained {len(df_pay_events_clean):,} payment events ({len(df_pay_events_clean)/len(df_pay_events)*100:.2f}% of original)")

---

## 11. Payments Receipts Table

### Evidence from Analysis

From `analysis.ipynb`:
- `brand_id`: 999,928 missing (90%)
- `price`: 14,892 missing (1.3%)

### Decision: Differential Strategy

**For brand_id (90% missing):** I impute with -1. The extreme missingness parallels Payments Events and suggests that receipt-level brand attribution is often unavailable in the transaction system. Receipts capture item-level details via `approximate_item_id`, making the transaction data useful even without brand mapping. Dropping 90% of receipt data would be unacceptable.

**For price (1.3% missing):** I drop rows with missing prices. Price is essential for receipt value calculation and revenue analysis. The 1.3% coverage loss is acceptable given the analytic importance of price integrity.

In [ ]:
# Load payments receipts
print("Loading payments receipts...")
receipts_dir = f"{DATASET_PATH_FULL}/payments/receipts"
receipt_files = dataset_files.get(receipts_dir, [])
df_receipts = load_dataframe_from_partitions_safe(
    receipt_files, 
    limit=DATASET_FULL_NUM_PARTITIONS_TO_LOAD
)

if df_receipts is not None:
    show_evidence(df_receipts, "Payments Receipts (Before Cleaning)", 
                 columns_to_check=['brand_id', 'price'])
    
    # Apply cleaning
    print("Applying differential strategy:")
    print("  - brand_id (90% missing): impute with -1")
    print("  - price (1.3% missing): drop rows")
    
    df_receipts_clean = df_receipts.copy()
    df_receipts_clean['brand_id'] = df_receipts_clean['brand_id'].fillna(-1)
    df_receipts_clean = df_receipts_clean.dropna(subset=['price'])
    
    # Validate
    validate_cleaning(df_receipts, df_receipts_clean, "Payments Receipts")
    save_cleaned_data(df_receipts_clean, "payments_receipts")
    print(f"Retained {len(df_receipts_clean):,} receipt records ({len(df_receipts_clean)/len(df_receipts)*100:.1f}% of original)")

---

## Reflection and Limitations

### What I Did Not Clean

Several common data cleaning steps were **intentionally omitted** because they lacked evidence:

1. **Outlier detection and removal**: The analysis showed no evidence of extreme or implausible values. Price distributions, while varied, appeared consistent with a diverse product catalog. Removing outliers without specific evidence would risk discarding legitimate premium or bulk-purchase items.

2. **Data type conversions**: All columns had appropriate types (`uint64` for IDs, `float64` for prices, `object` for text, `timedelta64` for timestamps). No conversions were necessary.

### Confirmed vs. Assumed

**Confirmed data errors** (with explicit evidence):
- Missing values (counts verified)
- Duplicate brand records (count verified)
- Embedding schema corruption (error message verified)

**Assumptions** (necessary but unverified):
- -1 does not conflict with legitimate ID values
- First-occurrence duplicate resolution is acceptable

### Cleaned Dataset Availability

All cleaned tables are saved in `cleaned_data/` as Parquet files, ready for modeling and analysis.